In [1]:
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt
import datetime
api = 'c4ad87b03bec1878bc0be6156d4472b0'
#pd.options.display.float_format = '{:,.2f}'.format

In [79]:
df_tech_sector_tickers = pd.read_excel('NASDAQ_tickers.xlsx', sheet_name='GICS - Sector')
df_tech_sector_tickers.head(), len(df_tech_sector_tickers)

(                       Company Ticker     Industry                    Sector
 0                      KBR INC    KBR  IT Services  IT Consulting & Services
 1                      INPIXON   INPX  IT Services  IT Consulting & Services
 2           AMERI HOLDINGS INC   AMRH  IT Services  IT Consulting & Services
 3  BOOZ ALLEN HAMILTON HLDG CP    BAH  IT Services  IT Consulting & Services
 4     COGNIZANT TECH SOLUTIONS   CTSH  IT Services  IT Consulting & Services,
 474)

In [80]:
tickers = df_tech_sector_tickers['Ticker']
tickers

0       KBR
1      INPX
2      AMRH
3       BAH
4      CTSH
       ... 
469    OLED
470    XLNX
471    EMAN
472    CEVA
473    RMBS
Name: Ticker, Length: 474, dtype: object

In [20]:
def growth(ticker):
    df = pd.read_csv(f'financials/{ticker}.csv')
    df.columns = df.columns.str.replace('Unnamed: 0', 'Date')
    df_margins = df[['Date','Ticker','Gross Profit Margin', 'Op Margin','Net Profit Margin']]
    df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
    df_margins['Op Margin'] = df_margins['Op Margin']*100
    df_margins['Net Profit Margin'] = df_margins['Net Profit Margin']*100
    return df_margins

In [21]:
growth('MSFT')

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/357065586.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/357065586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/357065586.py:7: SettingWithCopyWarning: 
A value is trying to be 

,Date,Ticker,Gross Profit Margin,Op Margin,Net Profit Margin
0,2022-09-24,MSFT,69.17,42.93,35.03
1,2022-06-25,MSFT,68.32,39.59,32.28
2,2022-03-26,MSFT,68.37,41.26,33.89
3,2021-12-25,MSFT,67.21,43.01,36.28
4,2021-09-25,MSFT,69.89,44.66,45.25
...,...,...,...,...,...
128,1990-09-30,MSFT,82.95,33.30,23.71
129,1990-06-30,MSFT,83.32,32.80,23.75
130,1990-03-31,MSFT,82.60,34.22,24.19
131,1989-12-31,MSFT,81.76,35.62,24.80


In [22]:
# working capital
# balance sheet
# TTM

In [67]:
def calendarize_dates(df):
    if df['Month']==9:
        return 3
    elif df['Month']==12:
        return 4
    elif df['Month']==1:
        return 4
    elif df['Month']==6:
        return 2
    elif df['Month']==7:
        return 2
    elif df['Month']==3:
        return 1
    elif df['Month']==4:
        return 1
    
        

In [72]:
def calendarize(ticker):
    if not os.path.exists(f'financials/{ticker}'):
       os.makedirs(f'financials/{ticker}')
    df = pd.read_csv(f'financials/{ticker}.csv')
    df.columns = df.columns.str.replace('Unnamed: 0', 'Date')
    df.columns = df.columns.str.replace('Quarter', 'Fiscal_Quarter')
    df.columns = df.columns.str.replace('Calendar_Year', 'Fiscal_Year')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    series = df.apply(calendarize_dates, axis=1)
    result = pd.concat((df, series.rename('Calendar_Quarter')), axis=1)
    result['Calendar_Year'] = result['Date'].dt.year
    result['Calendar_Quarter'] = result['Calendar_Quarter'].astype(int)
    
    # margins
    df_margins = result[['Date','Ticker','Calendar_Quarter', 'Calendar_Year','Gross Profit Margin', 'Op Margin','Net Profit Margin']]
    df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
    df_margins['Op Margin'] = df_margins['Op Margin']*100
    df_margins['Net Profit Margin'] = df_margins['Net Profit Margin']*100
    
    # growth
    df_growth = result[['Date', 'Ticker', 'Calendar_Quarter', 'Calendar_Year', 'Revenue','Gross Profit',
                      'Op Income', 'Net Income', 'NI per Share', 'Fcash_flow']]
    df_growth = df_growth[::-1]
    df_growth['Revenue Growth'] = df_growth['Revenue'].pct_change(periods=4)*100
    df_growth['Gross Profit Growth'] = df_growth['Gross Profit'].pct_change(periods=4)*100
    df_growth['Op Income Growth'] = df_growth['Op Income'].pct_change(periods=4)*100
    df_growth['Net Income Growth'] = df_growth['Net Income'].pct_change(periods=4)*100
    df_growth['EPS Growth'] = df_growth['NI per Share'].pct_change(periods=4)*100
    df_growth['FCF Growth'] = df_growth['Fcash_flow'].pct_change(periods=4)*100
    df_growth = df_growth[::-1]
    
    # Returns
    df_returns = result[['Date', 'Ticker', 'Month','Calendar_Quarter', 'Calendar_Year','ROA', 'ROE', 'ROCE']]
    df_returns['ROA'] = df_returns['ROA']*100
    df_returns['ROE'] = df_returns['ROE']*100
    df_returns['ROCE'] = df_returns['ROCE']*100
    
    # working capital
    
    ## balance sheet
    
    
    ## valuation
    df_val = result[['Date', 'Ticker', 'Calendar_Quarter', 'Calendar_Year','PE','PS','Price To FCF','PB']]
    
    
    ##############
    df_returns.to_csv(f'financials/{ticker}/df_returns_{ticker}.csv')
    df_growth.to_csv(f'financials/{ticker}/df_growth_{ticker}.csv')
    df_margins.to_csv(f'financials/{ticker}/df_margins_{ticker}.csv')
    df_val.to_csv(f'financials/{ticker}/df_val_{ticker}.csv')
    
    
        
    #return df_returns
    #return df_growth
    #return df_margins
    #return df_val
    #return result
    

In [73]:
aapl = calendarize('AAPL')
#aapl['Month'].value_counts()
aapl
msft = calendarize('MSFT')
msft


/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

,Date,Fiscal_Year,Fiscal_Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,...,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,Month,Calendar_Quarter,Calendar_Year
0,2022-09-24,2023,Q1,"1,736,735.25",0.33,0.52,6.72,2.35,"7,485.00","50,122.00",...,24.73,34.65,10.01,102.67,5.04,2.35,MSFT,9,3,2022
1,2022-06-25,2022,Q4,"1,919,547.32",0.35,0.54,6.94,2.24,"7,506.00","51,865.00",...,28.67,37.01,11.53,108.09,63.93,2.24,MSFT,6,2,2022
2,2022-03-26,2022,Q3,"2,310,166.81",0.37,0.53,6.59,2.23,"7,534.00","49,360.00",...,34.53,46.80,14.18,115.24,-3.20,2.23,MSFT,3,1,2022
3,2021-12-25,2022,Q2,"2,524,081.65",0.37,0.53,6.89,2.50,"7,555.00","51,728.00",...,33.63,48.80,15.77,292.99,-3.99,2.50,MSFT,12,4,2021
4,2021-09-25,2022,Q1,"2,118,065.06",0.40,0.55,6.03,2.73,"7,567.00","45,317.00",...,25.82,46.74,13.94,113.08,1.05,2.73,MSFT,9,3,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1990-09-30,1991,Q1,"8,166.88",0.00,0.18,0.04,0.01,"8,760.00",369.40,...,23.31,22.11,8.25,135.89,-3.88,0.01,MSFT,9,3,1990
129,1990-06-30,1990,Q4,"7,937.78",0.00,0.17,0.04,0.01,"7,520.00",336.90,...,24.81,23.56,8.64,"1,889.95",3.89,0.01,MSFT,6,2,1990
130,1990-03-31,1990,Q3,"5,783.61",0.00,0.19,0.04,0.01,"7,520.00",310.90,...,19.23,18.60,7.00,95.91,34.55,0.01,MSFT,3,1,1990
131,1989-12-31,1990,Q2,"4,591.58",0.00,0.21,0.04,0.01,"7,450.00",300.40,...,15.41,15.28,6.28,123.76,0.26,0.01,MSFT,12,4,1989


In [81]:
## call the fucntion to generate the files


#result = [calendarize(ticker) for ticker in tickers]

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

...

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_87744/1727214316.py:19: SettingWithCopyWarning: 
A value is trying 

In [ ]:
msft = calendarize('INTU')
msft.dtypes
msft

In [78]:
tickers

0       KBR
1      INPX
2      AMRH
3       BAH
4      CTSH
       ... 
470    OLED
471    XLNX
472    EMAN
473    CEVA
474    RMBS
Name: Ticker, Length: 475, dtype: object

In [88]:
import shutil

def move_files(ticker):

    source = '/Users/brendan/Desktop/Python/Software 2020/financials'
    destination = f'/Users/brendan/Desktop/Python/Software 2020/financials/{ticker}'
 
    # gather all files
    all_files = os.listdir(source)    
    ticker_file = list(filter(lambda f: f.endswith(f'{ticker}.csv'), all_files))
    # iterate on all files to move them to destination folder
    
    #return csv_files
    for f in ticker_file:
        src_path = os.path.join(source, f)
        dst_path = os.path.join(destination, f)
        shutil.move(src_path, dst_path)
    print('File moved')

In [91]:
# file cleanup

#result = [move_files(ticker) for ticker in tickers]

## To do

In [ ]:
## working capital metrics
# balance sheet metrics
# valuation with implied mutliples based on lag of 1 year

# compile into master df
# put into quintiles - qcut and percentle rank
# compare stock returns from pt in time using lag of 1 year, 3 yr, 5 yr
# find long-short spreads

# get all relevant tickers for GICS and broader tech

# cumsum on fundamentals

# north star: composite quant score/ top decile returns on capital